In [ ]:
import numpy as np
import nn.layers as layers
import nn.net as net
from utils import *

# Convolution layer
`nn/layers.py` 파일 내부에 있는 `Conv2d` 클래스의 `forward`와 `backward` 메소드를 주석에 명시된 요구 조건에 따라 작성하고, 아래 코드를 실행시켜 구현한 코드를 테스트한다.<br>
**NOTE**: 모든 테스트 결과는 *1e-06* 이하의 오차만 허용된다.

In [ ]:
# Test the Conv2d.forward()
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)

conv = layers.Conv2d(3, 3, 4, 2, 1)
conv.params["w"] = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
conv.params["b"] = np.linspace(-0.1, 0.2, num=3)

out = conv.forward(x)
correct_out = np.array([[[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]]])

print("Testing conv2d - forward function:")
print("error:", rel_error(out, correct_out))

In [ ]:
# Test the Conv2d.backward()
x = np.random.randn(4, 3, 5, 5)
conv = layers.Conv2d(3, 2, 3, 1, 1, init_mode="normal")
dout = np.random.randn(4, 2, 5, 5)

dx_num = eval_numerical_gradient_array(lambda x: conv.forward(x), x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv.forward(x), conv.params["w"], dout)
db_num = eval_numerical_gradient_array(lambda b: conv.forward(x), conv.params["b"], dout)

out = conv.forward(x)
conv.backward(dout)

print("Testing conv2d - backward function:")
print("dx error:", rel_error(conv.grads["x"], dx_num))
print("dw error:", rel_error(conv.grads["w"], dw_num))
print("db error:", rel_error(conv.grads["b"], db_num))

# Max pooling layer
`nn/layers.py` 파일 내부에 있는 `Maxpool2d` 클래스의 `forward`와 `backward` 메소드를 주석에 명시된 요구 조건에 따라 작성하고, 아래 코드를 실행시켜 구현한 코드를 테스트한다.<br>
**NOTE**: 모든 테스트 결과는 *1e-06* 이하의 오차만 허용된다.

In [ ]:
# Test the MaxPool2d.forward()
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
pool = layers.MaxPool2d(2, 2)
out = pool.forward(x)
correct_out = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])

print("Testing maxpool2d - forward function:")
print("error:", rel_error(out, correct_out))

In [ ]:
# Test the MaxPool2d.backward()
x = np.random.randn(3, 2, 8, 8)
dout = np.random.randn(3, 2, 4, 4)
pool = layers.MaxPool2d(2, 2)

dx_num = eval_numerical_gradient_array(lambda x: pool.forward(x), x, dout)
pool.forward(x)
pool.backward(dout)

print("Testing maxpool2d - backward function:")
print("dx error:", rel_error(pool.grads["x"], dx_num))

# Three-layer ConvNet
Convolution, pooling, FC 레이어를 종합하여 3-레이어 ConvNet을 구현한다. `nn/net.py`의 `FourlayerConvNet`를 작성한다.<br>
이 때, 네트워크의 구조는 아래와 같다.

- 3x3 conv - relu - 2x2 pool - fc - relu - fc - softmax

**NOTE**: Naive한 (loop 중첩) 방식의 구현은 매우 속도가 느려 NN 모델과 달리 학습하기가 어렵다. 따라서 본 과제에서는 구현한 convolution과 pooling 레이어의 결과 검증만 수행한다. 아래의 테스트 실행도 컴퓨팅 환경에 따라 최대 수 분이 소요될 수 있다.

In [ ]:
model = net.ThreeLayerConvNet(
    input_dim=(3,4,4), num_filters=8
)
X = np.random.randn(2, 3, 4, 4)
y = np.random.randint(10, size=2)

loss = model.loss(X, y)
print("Testing ThreeLayerConvNet:")
print("loss:", loss, " (must be around 2.3)")

# it takes long..
for module_name in model.modules.keys():
    if not model.modules[module_name].params:
        continue
        
    w = model.modules[module_name].params["w"]
    b = model.modules[module_name].params["b"]
    dw = model.modules[module_name].grads["w"]
    db = model.modules[module_name].grads["b"]
    
    f = lambda _: model.loss(X, y)
    grad_w = eval_numerical_gradient(f, w, verbose=False, h=1e-5)
    grad_b = eval_numerical_gradient(f, b, verbose=False, h=1e-5)

    print("{}_w error: {:e}".format(module_name, rel_error(dw, grad_w)))
    print("{}_b error: {:e}".format(module_name, rel_error(db, grad_b)))